# Model recovery attack: `wine`

In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import mra
from mra import *

In [2]:
def make_dataset(X, Y, f):
    x = tf.data.Dataset.from_tensor_slices(X)
    y = tf.data.Dataset.from_tensor_slices(Y)
    x = x.map(f)
    xy = tf.data.Dataset.zip((x, y))
    xy = xy.shuffle(10000)
    return xy

xy = np.loadtxt("../datasets/wine.csv", delimiter=";")
np.random.shuffle(xy)
x = xy[:,0:-1]
y = xy[:,-1]

min_values = np.array([min(x[:,i]) for i in range(x.shape[1])])
max_values = np.array([max(x[:,i]) for i in range(x.shape[1])])
x = (x-min_values)/(max_values - min_values)
y = y.reshape((len(x), 1)).astype("float32")
train_ds = make_dataset(x, y, lambda t: t)

mra = mra(train_ds)

2022-02-15 20:54:11.888403: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-15 20:54:15.708349: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2807 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:01:00.0, compute capability: 8.0


In [3]:
def make_f(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    x = tf.keras.layers.Dense(64, activation="relu")(xin)
    x = tf.keras.layers.Dense(64, activation="relu")(x)
    output = tf.keras.layers.Dense(64, activation="relu")(x)
    return tf.keras.Model(xin, output)

def make_g(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    x = tf.keras.layers.Dense(64, activation="relu")(xin)
    # x = tf.keras.layers.Dropout(0.5)(x)
    output = tf.keras.layers.Dense(11, activation="softmax")(x)
    return tf.keras.Model(xin, output)

In [4]:
log_train = mra.train(
    make_f,
    make_g,
    loss_fn=tf.keras.losses.sparse_categorical_crossentropy,
    batch_size=32,
    epoch=5,
    lr=0.001,
    verbose=True, log_every=50
    )

2022-02-15 20:59:12.737778: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Iteration 0001: Training loss: 2.3913
Iteration 0051: Training loss: 1.0352
Iteration 0101: Training loss: 1.2877
Iteration 0151: Training loss: 0.9521
Iteration 0201: Training loss: 0.9321


In [5]:
def make_generator(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    act = "relu"
    x = tf.keras.layers.Dense(512, activation=act)(xin)
    x = tf.keras.layers.Dense(256, activation=act)(x)
    x = tf.keras.layers.Dense(128, activation=act)(x)
    x = tf.keras.layers.Dense(64, activation=act)(x)
    x = tf.keras.layers.Dense(32, activation=act)(x)
    x = tf.keras.layers.Dense(11, activation="sigmoid")(x)
    return tf.keras.Model(xin, x)

In [6]:
log_attack = mra.attack(
    attack_iter=100,
    make_generator=make_generator,
    input_noise="normal",
    input_z=True,
    model_leak=True,
    lr_x=0.001,
    lr_f=0.0001,
    epoch=200,
    iter_x=50,
    iter_f=1,
    verbose=True, log_every=1)

Iteration 0001: RG-uniform: 0.1838, RG-normal: 0.1486, reconstruction validation: 0.0083
Iteration 0002: RG-uniform: 0.1753, RG-normal: 0.1622, reconstruction validation: 0.0082
Iteration 0003: RG-uniform: 0.1693, RG-normal: 0.1626, reconstruction validation: 0.0046
Iteration 0004: RG-uniform: 0.1542, RG-normal: 0.1617, reconstruction validation: 0.0028
Iteration 0005: RG-uniform: 0.1682, RG-normal: 0.1411, reconstruction validation: 0.0036
Iteration 0006: RG-uniform: 0.1673, RG-normal: 0.1713, reconstruction validation: 0.0034
Iteration 0007: RG-uniform: 0.1677, RG-normal: 0.1508, reconstruction validation: 0.0039
Iteration 0008: RG-uniform: 0.1778, RG-normal: 0.1535, reconstruction validation: 0.0060
Iteration 0009: RG-uniform: 0.1805, RG-normal: 0.1428, reconstruction validation: 0.0051
Iteration 0010: RG-uniform: 0.1583, RG-normal: 0.1672, reconstruction validation: 0.0054
Iteration 0011: RG-uniform: 0.1726, RG-normal: 0.1599, reconstruction validation: 0.0048
Iteration 0012: RG-un

In [7]:
np.savetxt("./log/wine-3.csv", log_attack, delimiter=",")